In [ ]:
import pandas as pd
import numpy as np
import tkinter as tk
import warnings
import locale
import os
from tkinter import messagebox
from datetime import datetime
import calendar

warnings.filterwarnings('ignore')
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

def obtener_fecha(mes_anterior=False):
    hoy = datetime.now()
    
    if mes_anterior:
        if hoy.month == 1:
            mes = 12
            año = hoy.year - 1
        else:
            mes = hoy.month - 1
            año = hoy.year
    else:
        mes = hoy.month
        año = hoy.year
    
    mes_nombre = calendar.month_abbr[mes].upper()[:3] # ENE
    mes_año = f"{mes_nombre}{str(año)[2:]}" # ENE24
    fecha = f"{año}{str(mes).zfill(2)}" # 202401
    
    return mes_año, fecha

mes_año, fecha = obtener_fecha(mes_anterior=True)

asignacion_path = f'bases/asignacion/{fecha}/base_asignacion_{mes_año}.txt'
recupero_path = f'bases/recupero/{fecha}/Recupero_{fecha}.csv'
cancelaciones_path = f'bases/cancelaciones/{fecha}/Cancelaciones_{mes_año}.xlsx'

stock_judicial_path = f'bases/recupero/{fecha}/StockJudicial.xlsx'
stock_castigo_path = f'bases/recupero/{fecha}/StockCastigos.xlsx'

final_path = os.path.abspath(cancelaciones_path)

print('Base Asignacion:', asignacion_path)
print('Base Recupero:',recupero_path)
print('Base Cancelaciones:', cancelaciones_path)
print('Stock Judicial:', stock_judicial_path)
print('Stock Castigo:', stock_castigo_path)

In [126]:
def clean_columns(columns_list: list[str]) -> list[str]:
    return [column.strip().replace('.', '').replace(' ', '_').upper() for column in columns_list]

In [ ]:
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Cargar stocks?', icon='warning')

if result == 'yes':
    # Stock Judicial
    df_cartera_judicial = pd.read_excel(stock_judicial_path)
    df_cartera_judicial.columns = clean_columns(df_cartera_judicial.columns)
    cols_judicial = ['CONTRATO', 'TIPO_CARTERA']
    df_cartera_judicial = df_cartera_judicial[cols_judicial]
    df_cartera_judicial['CONTRATO'] = df_cartera_judicial['CONTRATO'].apply(lambda x: x[-18:] if pd.notna(x) else x)
    print('Stock Judicial:', df_cartera_judicial.shape)
    # Stock Castigo
    df_cartera_castigo = pd.read_excel(stock_castigo_path)
    df_cartera_castigo.columns = clean_columns(df_cartera_castigo.columns)
    cols_castigo = ['CONTRATO', 'GRUPO']
    df_cartera_castigo = df_cartera_castigo[cols_castigo]
    df_cartera_castigo.rename(columns={'GRUPO': 'TIPO_CARTERA'}, inplace=True)
    df_cartera_castigo['CONTRATO'] = df_cartera_castigo['CONTRATO'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)
    print('Stock Castigo:', df_cartera_castigo.shape)

root.destroy()

In [ ]:
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Cargar base recupero?', icon='warning')

if result == 'yes':
    df_recupero = pd.read_csv(recupero_path, sep=';', encoding='utf-8')
    df_recupero.columns = clean_columns(df_recupero.columns)
    print('Base Recupero:', df_recupero.shape)

root.destroy()

In [ ]:
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Cargar base asignacion?', icon='warning')

if result == 'yes':
    df_asignacion = pd.read_csv(asignacion_path, sep='|', encoding='utf-8')
    df_asignacion.columns = clean_columns(df_asignacion.columns)
    print('Base Asignación:', df_asignacion.shape)

root.destroy()

In [ ]:
df_asignacion['TIPO_CARTERA'].value_counts(dropna=False)

In [ ]:
df_asignacion_test = df_asignacion.copy()

df_asignacion_test['CC'] = df_asignacion_test['CC'].astype('Int64').astype(str).str.zfill(8)
df_asignacion_test['CONTRATO'] = df_asignacion_test['CONTRATO'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)

df_asignacion_test['TIPO_FONDO'].replace({'REACTIVA_KST': 'REACTIVA'}, inplace=True)
df_asignacion_test['TIPO_FONDO'].fillna('NULL', inplace=True)

if df_asignacion_test['CAMP'].dtype == 'object':
    df_asignacion_test['CAMP'] = df_asignacion_test['CAMP'].str.extract(r'(\d+)%').astype(float) / 100
df_asignacion_test['CAMP'] = pd.to_numeric(df_asignacion_test['CAMP'], errors='coerce')
df_asignacion_test['CAMP'].fillna(0, inplace=True)

if df_asignacion_test['CAPITAL'].dtype == 'object':
    df_asignacion_test['CAPITAL'] = df_asignacion_test['CAPITAL'].str.replace(',', '').str.strip()
df_asignacion_test['CAPITAL'] = df_asignacion_test['CAPITAL'].replace('-', np.nan).astype(float)
df_asignacion_test['CAPITAL'].fillna(0, inplace=True)

df_asignacion_test['IMPORTE_CAMP'] = df_asignacion_test.apply(lambda x: x['CAPITAL'] if x['TIPO_FONDO'] in ['REACTIVA', 'FAE', 'CRECER'] else x['CAPITAL']*(1-x['CAMP']), axis=1)

cols_asignacion = ['AGENCIA', 'CC', 'CONTRATO', 'CARTERA', 'CAPITAL', 'IMPORTE_CAMP', 'TIPO_FONDO', 'PRODUCTO']
df_asignacion_test = df_asignacion_test[cols_asignacion]

print('Base Asignación:', df_asignacion_test.shape)
df_asignacion_test.head(5)

In [ ]:
df_recupero_test = df_recupero.copy()
df_cartera_judicial_test = df_cartera_judicial.copy()
df_cartera_castigo_test = df_cartera_castigo.copy()

df_recupero_test.rename(columns={'CODCENTRAL': 'CC', 'CONT_SAE': 'CONTRATO', 'ENTIDAD': 'CARTERA'}, inplace=True)

df_recupero_test = df_recupero_test[df_recupero_test['BANCA'] == 'MINORISTA']
df_recupero_test = df_recupero_test[df_recupero_test['TIPO_CARTERA'] != 'SECURED']

df_recupero_test['CARTERA'] = df_recupero_test['CARTERA'].replace('KST', 'KSTBC')
df_recupero_test['CC'] = df_recupero_test['CC'].astype('Int64').astype(str).str.zfill(8)
df_recupero_test['CONTRATO'] = df_recupero_test['CONTRATO'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)

df_recupero_test['TIPO_CARTERA'] = df_recupero_test.apply(lambda x: 'UNSECURED' if (x['CARTERA'] == 'KSTBC' and x['ENTIDAD_ORIGEN'] == 'EXJ') else x['TIPO_CARTERA'], axis=1)

df_recupero_test = df_recupero_test.merge(df_cartera_judicial_test, on='CONTRATO', how='left', suffixes=('', '_BC'))
df_recupero_test = df_recupero_test.merge(df_cartera_castigo_test, on='CONTRATO', how='left', suffixes=('', '_KSTBC'))

df_recupero_test['TIPO_CARTERA'] = np.where((df_recupero_test['CARTERA'] == 'EXJ'), 'UNSECURED', df_recupero_test['TIPO_CARTERA'])

df_recupero_test['TIPO_CARTERA'] = df_recupero_test['TIPO_CARTERA'].str.strip()
df_recupero_test['TIPO_CARTERA'].fillna('NULL', inplace=True)

df_recupero_test['TIPO_CARTERA_BC'] = df_recupero_test['TIPO_CARTERA_BC'].str.strip()
df_recupero_test['TIPO_CARTERA_BC'].fillna('NULL', inplace=True)

df_recupero_test['TIPO_CARTERA_KSTBC'] = df_recupero_test['TIPO_CARTERA_KSTBC'].str.strip()
df_recupero_test['TIPO_CARTERA_KSTBC'].fillna('NULL', inplace=True)

df_recupero_test[['CARTERA', 'TIPO_CARTERA']].value_counts(dropna=False).sort_index()

In [ ]:
def validar_tipo_cartera(row):
    tipo_cartera = row['TIPO_CARTERA']
    
    if row['CARTERA'] == 'BC':
        tipo_cartera_bc = row['TIPO_CARTERA_BC']
        if tipo_cartera_bc == 'NULL' or tipo_cartera == tipo_cartera_bc:
            return tipo_cartera
        else:
            return f'{tipo_cartera}/{tipo_cartera_bc}'
    
    elif row['CARTERA'] == 'KSTBC':
        tipo_cartera_kstbc = row['TIPO_CARTERA_KSTBC']
        if tipo_cartera_kstbc == 'NULL' or tipo_cartera == tipo_cartera_kstbc:
            return tipo_cartera
        else:
            return f'{tipo_cartera}/{tipo_cartera_kstbc}'
    
    else:
        return tipo_cartera

df_recupero_test['VALIDADOR'] = df_recupero_test.apply(validar_tipo_cartera, axis=1)

df_recupero_test[['CARTERA', 'VALIDADOR']].value_counts(dropna=False).sort_index()

In [ ]:
df_recupero_test['LIQUIDO'].fillna(0, inplace=True)
df_recupero_test['TRANSACCION_REFINANCIADO'].fillna(0, inplace=True)
df_recupero_test['TOTAL'] = df_recupero_test['LIQUIDO'] + df_recupero_test['TRANSACCION_REFINANCIADO']

df_grouped = df_recupero_test.groupby('CONTRATO')['TOTAL'].sum().reset_index()

df_recupero_test = df_recupero_test.drop_duplicates(subset='CONTRATO', keep='last')
df_recupero_test = df_recupero_test.merge(df_grouped, on='CONTRATO', suffixes=('', '_sum'))

df_recupero_test['TOTAL'] = df_recupero_test['TOTAL_sum']
df_recupero_test.drop(columns='TOTAL_sum', inplace=True)

df_recupero_test = df_recupero_test[df_recupero_test['TOTAL'] > 0]
df_recupero_test.sort_values(['CC', 'CONTRATO', 'CARTERA', 'TOTAL'], inplace=True)
df_recupero_test.reset_index(drop=True, inplace=True)

cols_recupero = ['CC', 'CONTRATO', 'CARTERA', 'TOTAL', 'BANCA', 'TIPO_CARTERA', 'VALIDADOR']
df_recupero_test = df_recupero_test[cols_recupero]

print('Base Recupero:', df_recupero_test.shape)
df_recupero_test.head(5)

In [ ]:
df_recupero_test[['CARTERA', 'VALIDADOR']].value_counts(dropna=False).sort_index()

In [ ]:
df_cancelaciones = df_recupero_test.merge(df_asignacion_test[['CONTRATO', 'IMPORTE_CAMP', 'CAPITAL', 'TIPO_FONDO', 'PRODUCTO']], on='CONTRATO', how='left')

df_cancelaciones['CUMPLE'] = np.where(df_cancelaciones['TOTAL'] >= df_cancelaciones['IMPORTE_CAMP'], 'SI', 'NO')
df_cancelaciones['TIPO_PAGO'] = np.where(
    df_cancelaciones['TIPO_FONDO'].isin(['REACTIVA', 'FAE' , 'CRECER']),
    'PAGO TOTAL', 
    np.where(
        df_cancelaciones['TOTAL'] >= df_cancelaciones['CAPITAL'], 
        'PAGO CAPITAL', 
        'PAGO CAMPAÑA'
    )
)
df_cancelaciones['TOTAL'] = df_cancelaciones['TOTAL'].round(2)
df_cancelaciones['IMPORTE_CAMP'] = df_cancelaciones['IMPORTE_CAMP'].round(2)

df_cancelaciones = df_cancelaciones[df_cancelaciones['CUMPLE'] == 'SI']

df_cancelaciones.rename(columns={'TOTAL': 'IMPORTE_SUMA', 'IMPORTE_CAMP': 'IMPORTE_MINIMO'}, inplace=True)
df_cancelaciones = df_cancelaciones[df_cancelaciones['IMPORTE_SUMA'] > 0]
df_cancelaciones = df_cancelaciones[df_cancelaciones['VALIDADOR'] != 'UNSECURED/SECURED']

df_cancelaciones['OBSERVACIONES'] = df_cancelaciones.apply(lambda x: 'SIN MARCA UNSECURED' if x['VALIDADOR'] == 'NULL' else '', axis=1)
df_cancelaciones[['CARTERA', 'VALIDADOR', 'OBSERVACIONES']].value_counts(dropna=False).sort_index()

In [ ]:
df_cancelaciones.groupby(['TIPO_FONDO', 'TIPO_PAGO'])['IMPORTE_SUMA'].sum().sort_index(ascending=False)

In [ ]:
df_cancelaciones = df_cancelaciones[['CC', 'CONTRATO', 'CARTERA', 'IMPORTE_SUMA', 'IMPORTE_MINIMO', 'CUMPLE', 'TIPO_FONDO', 'BANCA', 'TIPO_CARTERA', 'PRODUCTO', 'TIPO_PAGO', 'OBSERVACIONES']]
df_cancelaciones.sort_values(['IMPORTE_SUMA' ,'TIPO_FONDO'], ascending=False, inplace=True)
df_cancelaciones.reset_index(drop=True, inplace=True)
df_cancelaciones.to_excel(final_path, index=False, sheet_name=f'CANCELACIONES {mes_año}')

print('Base Cancelaciones:', df_cancelaciones.shape)
df_cancelaciones.head(5)

In [115]:
import openpyxl as op
from openpyxl.styles import Font, PatternFill, Alignment

def format_excel(file_path: str) -> None:
    workbook = op.load_workbook(file_path)
    sheet = workbook.active
    
    general_font = Font(name='Calibri', size=11)
    header_font = Font(name='Calibri', size=11, bold=True, color='FFFFFF')
    alignment_center = Alignment(horizontal='center', vertical='center')
    
    header_fill_blue = PatternFill(start_color='0F243E', end_color='0F243E', fill_type='solid')
    header_fill_skyblue = PatternFill(start_color='0070C0', end_color='0070C0', fill_type='solid')
    
    sheet.row_dimensions[1].height = 50
    
    for row in sheet.iter_rows():
        for cell in row:
            cell.font = general_font
    
    for cell in sheet[1]:
        cell.font = header_font
        cell.alignment = alignment_center
    
    for col in range(1, 13):  # Columnas A-L
        sheet.cell(row=1, column=col).fill = header_fill_blue
    for col in range(10, 12):  # Columnas J-K
        sheet.cell(row=1, column=col).fill = header_fill_skyblue
    
    for column in sheet.columns:
        max_length = 0
        column_letter = column[0].column_letter
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2)
        sheet.column_dimensions[column_letter].width = adjusted_width
    
    workbook.save(file_path)

format_excel(final_path)
os.startfile(final_path)